In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logistic').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [4]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
log_model = LogisticRegression()

In [6]:
fitted_log_reg = log_model.fit(my_data)

In [7]:
log_summary = fitted_log_reg.summary

In [9]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [10]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [11]:
log_summary.predictions.select('label','prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [12]:
lr_train, lr_test = my_data.randomSplit([0.7,0.3])

In [13]:
final_model = LogisticRegression()

In [14]:
fit_final = final_model.fit(lr_train)

In [15]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [16]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[23.2435424872038...|[0.99999999991956...|       0.0|
|  0.0|(692,[124,125,126...|[29.0613422834780...|[0.99999999999976...|       0.0|
|  0.0|(692,[124,125,126...|[31.2820607702387...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[25.0724304726490...|[0.99999999998708...|       0.0|
|  0.0|(692,[126,127,128...|[16.3123709681994...|[0.99999991765689...|       0.0|
|  0.0|(692,[127,128,129...|[24.2535079907624...|[0.99999999997070...|       0.0|
|  0.0|(692,[127,128,129...|[19.6181818292484...|[0.99999999698052...|       0.0|
|  0.0|(692,[128,129,130...|[19.7199918822193...|[0.99999999727280...|       0.0|
|  0.0|(692,[154,155,156...|[18.0420751924939...|[0.99999998539753...|       0.0|
|  1.0|(692,[99,

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [18]:
my_eval = BinaryClassificationEvaluator()

In [19]:
final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [20]:
final_roc

1.0

In [21]:
#means it was 100% correct